In [1]:
import PyQt5
from PyQt5 import Qt
Qt.QT_VERSION_STR
FORCE_ENABLE_INLINE = False

In [2]:
import os
import sys
import gin
import numpy as np
import pandas as pd
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import matplotlib
matplotlib.use('Qt5Agg')

%matplotlib qt
#%matplotlib inline

import matplotlib.pyplot as plt
#import matplotlib.pyplot as plt
    
%load_ext autoreload
%autoreload 2

In [6]:
params = {
                    "sep": '\s+',
                   # "nrows": 10000,
                    "encoding": 'utf-8',
                   "names":  ['event',  'ntrack', 'det','station', 'side', 'z', 'y', 'x', 'time','dX', 'dY', 'dZ', 'dTime']
                 }
#data = pd.read_csv("../data_cbm/hits10_withStations.txt", **params)
data = pd.read_csv("../data_cbm/hits14/hits11.txt", **params)
data

,event,ntrack,det,station,side,z,y,x,time,dX,dY,dZ,dTime
0,0,1,0,0,0,189.100000,-6.464354,34.048732,-4.500000,0.000000,0.173103,0.173103,4.000000
1,0,1,0,0,0,189.100000,-9.437009,19.564944,-2.500000,0.000000,0.108496,0.108496,4.000000
2,0,1,0,0,0,189.100000,-2.309983,48.091135,-1.500000,0.000000,0.240480,0.240480,4.000000
3,0,1,0,0,0,189.100000,-3.327034,33.040807,1.500000,0.000000,0.117551,0.117551,2.833316
4,0,1,0,0,0,189.100000,-3.364102,24.718951,2.500000,0.000000,0.124603,0.124603,4.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2591988,999,1,2,0,0,853.570007,92.689089,38.896963,29.488937,0.288675,0.135316,0.720000,0.080000
2591989,999,1,2,0,0,862.650024,96.190628,67.393342,29.281079,0.288675,0.135316,0.720000,0.080000
2591990,999,1,2,0,0,853.570007,160.754187,66.451325,40.085243,0.288675,0.135316,0.720000,0.080000
2591991,999,1,2,0,0,853.570007,-68.090622,83.204689,28.777892,0.288675,0.135316,0.720000,0.080000


In [7]:
print(data.station.unique(),
      data.det.unique(),
      data.side.unique(),
     data.ntrack.unique())

[ 0  1  2  3  4  5  6  7  8  9 10 11] [0 1 2] [0 1] [1 2 3 4]


In [12]:
def visualize_simple_3d(data, axes=['x','y','z'], 
                        titleAx=['X', 'Y', 'Z'], 
                        title=None, 
                        scatter_params=None,
                        col_arr=None,
                        custom_draw=None):
    fig = plt.figure(figsize=(6,4))
    ax = fig.add_subplot(111, projection='3d')
    if custom_draw is not None:
        custom_draw(ax)
    else:
        if not scatter_params:
            scatter_params = {}
        if col_arr is None:        
            ax.scatter(data[axes[2]], data[axes[1]], data[axes[0]], **scatter_params)
        else:
            ax.scatter(data[axes[2]], data[axes[1]], data[axes[0]], c=col_arr, **scatter_params)
    ax.set_xlabel(titleAx[2])
    ax.set_ylabel(titleAx[1])
    ax.set_zlabel(titleAx[0])
    if title:
        plt.title(title)
    plt.tight_layout()
    plt.show()

[ 0  1  2  3  4  5  6  7  8  9 10 11] [0 1 2] [0 1] [0 1 2 3]


In [7]:
# max_stat = data.station.max()
# max_side = data.side.max()

# data['colr'] = (data.station.values * (1./max_stat) ) 
# data['colg'] = 0 * (data.side.values * (1./max_side) )
# data['colb'] = 0 * 0.5 * data.colr.values

true_events = data[data.ntrack >= 2]
false_events = data[data.ntrack < 2]

one_true_event = true_events[true_events.event == true_events.event.unique()[0]]
one_false_event = false_events[false_events.event == false_events.event.unique()[0]]

In [8]:
visualize_simple_3d(one_true_event, title='True event', 
                    scatter_params={'s':10},
                    col_arr=one_true_event[['colr', 'colg', 'colb']].values
                   )
#visualize_simple_3d(one_false_event, title='False event', scatter_params={'c':'tab:red', 's':5})

KeyError: "None of [Index(['colr', 'colg', 'colb'], dtype='object')] are in the [columns]"

In [15]:
def custom_draw(ax, event=None):
    #event = event[(event.station == 11) | (event.station == 10)| (event.station == 0)]
    for station_id, by_station in event.groupby('station'):
        for side_id, by_side in by_station.groupby('side'):
            for det_id, by_det in by_side.groupby('det'):
                lab = "det #%d station #%d side #%d" % (det_id, station_id, side_id)
                ax.scatter(by_det.z, by_det.y, by_det.x, label=lab, s=35)
    #ax.legend(bbox_to_anchor=(1.1, 1.05))

from functools import partial


visualize_simple_3d(None, title='True event det0', 
                    custom_draw=partial(custom_draw, event=one_true_event[one_true_event.det == 0]))
visualize_simple_3d(None, title='True event det1',
                    custom_draw=partial(custom_draw, event=one_true_event[one_true_event.det == 1]))
visualize_simple_3d(None, title='True event det2',
                    custom_draw=partial(custom_draw, event=one_true_event[one_true_event.det == 2]))



In [16]:
def custom_draw(ax, event=None):
    #event = event[(event.station == 11) | (event.station == 10)| (event.station == 0)]
    for station_id, by_station in event.groupby('station'):
        for side_id, by_side in by_station.groupby('side'):
            for det_id, by_det in by_side.groupby('det'):
                lab = "det #%d station #%d side #%d" % (det_id, station_id, side_id)
                ax.scatter(by_det.z, by_det.y, by_det.x, label=lab, s=35)
    #ax.legend(bbox_to_anchor=(1.1, 1.05))

from functools import partial


visualize_simple_3d(None, title='False event det0', 
                    custom_draw=partial(custom_draw, event=one_false_event[one_false_event.det == 0]))
visualize_simple_3d(None, title='False event det1',
                    custom_draw=partial(custom_draw, event=one_false_event[one_false_event.det == 1]))
visualize_simple_3d(None, title='False event det2',
                    custom_draw=partial(custom_draw, event=one_false_event[one_false_event.det == 2]))

